In [47]:
import os
import PIL.Image as Image
import torch.cuda
import torch.nn as nn
import torchvision.transforms as transforms
import numpy as np
import torchvision
import LookGenerator.datasets.transforms as custom_transforms
from LookGenerator.datasets.utils import prepare_images_for_encoder, load_image
from LookGenerator.networks.encoder_decoder import EncoderDecoder
from LookGenerator.networks.clothes_feature_extractor import ClothAutoencoder
from LookGenerator.networks.utils import load_model
import cv2
from tqdm import tqdm


# Загрузка изображений

In [85]:
#root = r"C:\Users\Даша\кто\мусор\zalando-hd-resized\val"
root = r"C:\Users\DenisovDmitrii\Desktop\forEncoderNew\val"
file_name = r"00006_00"

human_image = load_image(root, r"imageWithNoCloth", file_name, ".png")
clothes = load_image(root, r"cloth", file_name, r".jpg")

# pose_points = []
# points_list = os.listdir(os.path.join(
#     root,
#     r"posePoints",
#     file_name
# ))
# print(len(points_list))
# for pose_point in points_list:
#     pose_point_image = convert_channel(load_image(root, os.path.join(r"posePoints", file_name), pose_point, ""))
#     pose_points.append(pose_point_image)

# if model dataset has pose_points=False param:
# pose_points = []

# Определение трансформаций

In [86]:
transform_human = transforms.Compose([
    transforms.Resize((256, 192)) #,
    # transforms.RandomAffine(scale=(0.8, 1), degrees=(-90,90), fill = 0.9),
    # transforms.ColorJitter(brightness=(0.5, 1), contrast=(0.4,1),  hue=(0, 0.3)),
    # transforms.Normalize(mean=[0.5, 0.5, 0.5],
    #                      std=[0.5, 0.5, 0.5])
])

transform_clothes = transforms.Compose([
    transforms.Resize((256, 192)),
    # transforms.ColorJitter(brightness=(0.5, 1), contrast=(0.4,1),  hue=(0, 0.3)),
    transforms.Normalize(mean=[0.5, 0.5, 0.5],
                         std=[0.5, 0.5, 0.5])
])

# input_bin_transform = transforms.Compose([
#     transforms.Resize((256, 192)),
#     custom_transforms.Normalize()
# ])

In [87]:
totensor = transforms.ToTensor()
human_image = transform_human(totensor(human_image))
clothes_image = transform_clothes(totensor(clothes))

# Загрузка весов модели

In [88]:
model_input = torch.cat((human_image, clothes_image), axis=0)


In [89]:
model_input.shape

torch.Size([6, 256, 192])

In [90]:
model_input = model_input.unsqueeze(0)

In [91]:
model_input.shape


torch.Size([1, 6, 256, 192])

In [92]:
# model_input = prepare_images_for_encoder(human_image, pose_points, clothes, input_rgb_transform, input_bin_transform).float()

In [93]:
#weights_dir = r"C:\Users\Даша\PycharmProjects\SMBackEnd\LookGenerator\weights\epoch_19.pt"
weights_dir = r"C:\Users\DenisovDmitrii\OneDrive - ITMO UNIVERSITY\peopleDetector\newEncoder\weights\testWithMask_weights102\epoch_09.pt"

In [94]:
clothes_feature_extractor = ClothAutoencoder(
    in_channels=3,
    out_channels=3,
    features=(8, 16, 32, 64),
    latent_dim_size=128,
    encoder_activation_func=nn.LeakyReLU(),
    decoder_activation_func=nn.ReLU()
)
clothes_feature_extractor = load_model(clothes_feature_extractor, r"C:\Users\DenisovDmitrii\OneDrive - ITMO UNIVERSITY\peopleDetector\autoDegradation\weights\testClothes_L1Loss_4features\epoch_39.pt")

In [95]:
model = EncoderDecoder(clothes_feature_extractor, in_channels=6, out_channels=3)
model = load_model(model, weights_dir)

In [96]:
device = 'cpu' if torch.cuda.is_available() else 'cpu'

model = model.to(device)
model_input = model_input.to(device)

# Прогон модели

In [97]:
save_path = fr"C:\Users\DenisovDmitrii\Desktop\forEncoderNew\{file_name}.png"
save_path_old = fr"C:\Users\DenisovDmitrii\Desktop\forEncoder\out\{file_name}_old.png"

In [98]:
model_output = model(model_input)

In [99]:
torchvision.utils.save_image(model_output, save_path)

# Отображение результата

In [40]:
model_output = to_array_from_decoder(model_output)

In [41]:
to_plt = np.array(model_output)

In [42]:
c = np.array(to_plt[:,:,0])
to_plt[:,:,0] = to_plt[:,:,2]
to_plt[:,:,2] = c

In [ ]:
show_array_as_image(np.invert(np.uint8(255*to_plt)))
#save_array_as_image(np.uint8(255*model_output), save_path=save_path_old)
# cv2.imwrite(fr"C:\Users\DenisovDmitrii\Desktop\forEncoder\out\{file_name}.png", cv2.cvtColor(np.uint8(255*model_output), cv2.COLOR_RGB2BGR) )

In [125]:
dir_ = r"C:\Users\DenisovDmitrii\Desktop\forEncoder\val\imageWithNoCloth"
list_files = os.listdir(dir_)

In [126]:
input_rgb_transform = transforms.Compose([
    transforms.Resize((256, 192)),
    custom_transforms.Normalize()
])

input_bin_transform = transforms.Compose([
    transforms.Resize((256, 192)),
    custom_transforms.Normalize()
])

In [127]:
weights_dir = r"C:\Users\DenisovDmitrii\OneDrive - ITMO UNIVERSITY\peopleDetector\encoder\weights\session19_2\epoch_01.pt"
model = EncoderDecoder(in_channels=6, out_channels=3)
model = load_model(model, weights_dir)
device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = model.to(device)


In [ ]:
for file_name_ in tqdm(list_files[:150]):
    root = r"C:\Users\DenisovDmitrii\Desktop\forEncoder\val"
    file_name = file_name_[:-4]

    human_image = load_image(root, r"imageWithNoCloth", file_name, ".png")
    clothes = load_image(root, r"cloth", file_name, r".jpg")

    pose_points = []
    points_list = os.listdir(os.path.join(
        root,
        r"posePoints",
        file_name
    ))
    for pose_point in points_list:
        pose_point_image = convert_channel(load_image(root, os.path.join(r"posePoints", file_name), pose_point, ""))
        pose_points.append(pose_point_image)

    # if model dataset has pose_points=False param:
    pose_points = []
    model_input = prepare_images_for_encoder(human_image, pose_points, clothes, input_rgb_transform, input_bin_transform).float()
    model_input = model_input.to(device)

    save_path = fr"C:\Users\DenisovDmitrii\Desktop\forEncoder\out\{file_name}.png"
    model_output = model(model_input)
    torchvision.utils.save_image(model_output.to('cpu'), save_path)
